## KFall 데이터셋

허리 뒷부분에 부착된 관성 센서를 통해 32명의 젊은 피실험자가 21가지 일상생활 동작(ADL)과 15가지 타입의 넘어짐 동작을 수행하는 동안 수집됨. 총 5075개의 동작 파일이 있으며, 이 중 2729개는 ADL 동작이고, 2346개는 넘어짐 동작임. 각각의 넘어짐 동작에 대해 동기화된 비디오 참조를 기반으로 한 준비된 넘어짐 라벨(넘어짐 시작 및 넘어짐 충격 시점)도 포함되어 있음.

 - 센서 데이터의 구성:
sensor_data.zip 파일에는 32개의 서브 폴더가 있으며, 각각은 주체 ID로 명명되어 있음. (예: SA06).
각 서브 폴더에는 해당 주체의 모든 동작 파일이 포함되어 있음.
동작 파일의 명명 규칙은 다음과 같이 설명됨. 예를 들어, "SA06T01R01.csv"는 "SA06"이라는 주체 ID를 가지며, "T01"은 작업 ID를 의미하고, "R01"은 시행 ID를 의미함. 작업 01, 11, 12 및 17을 제외한 작업은 일반적으로 5회의 시행이 필요하지만, 일부 작업은 Bluetooth 연결 불량, 신호 동기화 및 오류로 인해 4회 또는 6회의 시행을 가진 데이터도 있음. (참고: SA01~SA05는 시험 용으로 사용된 주체로, 제외. 시스템 오류로 인해 SA34의 데이터가 누락.).
각 동작 파일(.csv)은 11개의 열로 구성되어 있으며, 시간스탬프(초), 프레임 카운터, 가속도(단위: g), 자이로스코프(단위: °/s) 및 세 축을 따라 오일러 각도(°)를 포함.

 - 레이블 데이터의 구성:
label_data.zip 파일에는 32개의 주체 ID로 명명된 레이블 파일이 있으며, 각 레이블 파일에는 6개의 열이 있음: 작업 코드 (작업 ID), 설명, 시행 ID, 추락 시작 프레임 및 추락 충격 프레임. 마지막 두 열은 동기화된 비디오 참조를 기반으로 함. SA06_label.xlsx의 첫 번째 레코드를 예로 들면, 해당 데이터 파일 (SA06T20R01.csv)에서 추락 시작 프레임은 130이고 추락 충격 프레임은 208.



In [ ]:
import pandas as pd
import os
import re

# 센서 데이터와 레이블 데이터 경로 설정
sensor_data_path = '../data/sensor_data'
label_data_path = '../data/label_data'

# 센서 데이터 로드 및 전처리
sensor_data = pd.DataFrame()
for root, dirs, files in os.walk(sensor_data_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            subject_id = file.split('T')[0]  # 파일명에서 주체 ID 추출
            task_code = re.sub(r'\D', '', file.split('T')[1].split('R')[0])  # 파일명에서 태스크 코드 추출 (숫자만 추출)
            trial_id = file.split('R')[1].split('.')[0]  # 파일명에서 트라이얼 ID 추출
            df['Subject ID'] = subject_id
            df['Task Code'] = task_code
            df['Trial ID'] = trial_id
            sensor_data = pd.concat([sensor_data, df], ignore_index=True)

print(sensor_data)

# # 레이블 데이터 로드 및 전처리
# label_data = pd.DataFrame()
# for root, dirs, files in os.walk(label_data_path):
#     for file in files:
#         if file.endswith('.xlsx'):
#             file_path = os.path.join(root, file)
#             df = pd.read_excel(file_path)
#             subject_id = file.split('_')[0]  # 파일명에서 주체 ID 추출
#             df['Subject ID'] = subject_id
#             df.rename(columns={'Task Code (Task ID)': 'Task Code'}, inplace=True)  # 컬럼명 수정
#             label_data = pd.concat([label_data, df], ignore_index=True)
#
# # 결측값 처리
# label_data[['Description', 'Task Code']] = label_data[['Description', 'Task Code']].fillna(method='ffill')
#
# # 데이터 타입 변환
# sensor_data['Subject ID'] = sensor_data['Subject ID'].astype(str)
# sensor_data['Task Code'] = sensor_data['Task Code'].astype(int)
# sensor_data['Trial ID'] = sensor_data['Trial ID'].astype(int)
# label_data['Subject ID'] = label_data['Subject ID'].astype(str)
# label_data['Task Code'] = label_data['Task Code (Task ID)'].str.extract(r'(\d+)').astype(int)  # 숫자만 추출하여 정수로 변환
# label_data['Trial ID'] = label_data['Trial ID'].astype(int)
#
# # 센서 데이터와 레이블 데이터 결합
# merged_data = pd.merge(sensor_data, label_data, on=['Subject ID', 'Task Code', 'Trial ID'])
#
# # 결과 출력
# print(merged_data.head())
# print(label_data)


In [1]:
import os
import re
import pandas as pd

#### 센서 데이터를 로드하고 전처리.
os.walk() 함수를 사용하여 sensor_data_path에서 모든 파일을 찾아서 각 파일을 반복하면서 확장자가 '.csv'인 파일을 확인. 해당 파일을 DataFrame으로 로드하고 주체 ID, 태스크 코드, 트라이얼 ID를 추출하여 각각 열로 추가. pd.concat() 함수를 사용하여 모든 데이터를 하나의 DataFrame으로 병합.

In [8]:
sensor_data_path = '../data/sensor_data'

sensor_data = pd.DataFrame()
for root, dirs, files in os.walk(sensor_data_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            subject_id = file.split('T')[0]  # 파일명에서 주체 ID 추출
            task_id = re.sub(r'\D', '', file.split('T')[1].split('R')[0])  # 파일명에서 태스크 ID 추출 (숫자만 추출)
            trial_id = file.split('R')[1].split('.')[0]  # 파일명에서 트라이얼 ID 추출
            df['Subject ID'] = subject_id
            df['Task ID'] = task_id
            df['Trial ID'] = trial_id
            sensor_data = pd.concat([sensor_data, df], ignore_index=True)

print(sensor_data)

         TimeStamp(s)  FrameCounter   AccX   AccY   AccZ      GyrX      GyrY  \
0                0.01             1 -0.053 -1.004  0.000 -0.057296  0.401071   
1                0.02             2 -0.054 -1.007  0.000 -0.057296  0.229183   
2                0.03             3 -0.056 -1.009 -0.004 -0.458366  0.286479   
3                0.04             4 -0.053 -1.011 -0.009 -0.916733  0.458366   
4                0.05             5 -0.052 -1.012 -0.011 -1.088620  0.343775   
...               ...           ...    ...    ...    ...       ...       ...   
3995095          8.16           816 -0.035 -1.004  0.138 -0.229183 -1.661578   
3995096          8.17           817 -0.030 -1.000  0.135 -0.630254 -1.375099   
3995097          8.18           818 -0.024 -0.997  0.131 -1.203212 -1.546987   
3995098          8.19           819 -0.019 -0.997  0.126 -1.776170 -1.776170   
3995099          8.20           820 -0.014 -0.999  0.120 -2.234536 -2.234536   

             GyrZ     EulerX    EulerY 

label_dict = {
    '01': '30초 동안 서기',
    '02': '서서 천천히 등을 구부리거나 무릎을 구부리지 않고 신발 끈을 묶고 일어나기',
    '03': '바닥에서 물건 집어올리기',
    '04': '부드럽게 점프하기 (물체에 닿아보려고 시도)',
    '05': '서서 바닥에 앉아 잠시 기다린 후 정상 속도로 일어나기',
    '06': '보행하면서 정상적으로 회전하기 (4m)',
    '07': '보행하면서 빠른 속도로 회전하기 (4m)',
    '08': '정상 속도로 조깅하면서 회전하기 (4m)',
    '09': '빠른 속도로 조깅하면서 회전하기 (4m)',
    '10': '걷다가 비틀거리기',
    '11': '의자에 30초 동안 앉기',
    '12': '소파에 기댄 상태로 30초 동안 앉기',
    '13': '정상적으로 의자에 앉고 정상적으로 일어나기',
    '14': '빠르게 의자에 앉고 빠르게 일어나기',
    '15': '잠시 앉아 일어나려고 하다 의자에 쓰러지기',
    '16': '서서 소파에 앉고 정상적으로 일어나기',
    '17': '침대에 누워 30초 동안 머무르기',
    '18': '잠시 앉아 정상적으로 침대에 눕고 정상적으로 일어나기',
    '19': '잠시 앉아 빠르게 침대에 눕고 빠르게 일어나기',
    '20': '정상적으로 계단 오르내리기 (5단계)',
    '21': '빠른 속도로 계단 오르내리기 (5단계)',
    '22': '앉으려고 할 때 앞으로 넘어지기',
    '23': '앉으려고 할 때 뒤로 넘어지기',
    '24': '앉으려고 할 때 옆으로 넘어지기',
    '25': '일어나려고 할 때 앞으로 넘어지기',
    '26': '일어나려고 할 때 옆으로 넘어지기',
    '27': '앉아 있는 상태에서 실신으로 앞으로 넘어지기',
    '28': '앉아 있는 상태에서 실신으로 옆으로 넘어지기',
    '29': '앉아 있는 상태에서 실신으로 뒤로 넘어지기',
    '30': '걷다가 실신으로 앞으로 (수직으로) 넘어지기',
    '31': '걷다가 실신으로 넘어지지만 손을 이용하여 충격을 흡수하기',
    '32': '걷다가 걸려 넘어지기',
    '33': '조깅하다가 걸려 넘어지기',
    '34': '걷다가 미끄러져 넘어지기',
    '35': '걷다가 옆으로 미끄러져 넘어지기',
    '36': '걷다가 뒤로 미끄러져 넘어지기'
}

In [9]:
# 센서 데이터에 라벨을 추가할 라벨링 정보 딕셔너리 생성
label_dict = {
    '01': 'Stand for 30 seconds',
    '02': 'Stand, slowly bend the back with or without bending at knees, tie shoe lace, and get up',
    '03': 'Pick up an object from the floor',
    '04': 'Gently jump (try to reach an object)',
    '05': 'Stand, sit to the ground, wait a moment, and get up with normal speed',
    '06': 'Walk normally with turn (4m)',
    '07': 'Walk quickly with turn (4m)',
    '08': 'Jog normally with turn (4m)',
    '09': 'Jog quickly with turn (4m)',
    '10': 'Stumble while walking',
    '11': 'Sit on a chair for 30 seconds',
    '12': 'Sit on the sofa (back is inclined to the support) for 30 seconds',
    '13': 'Sit down to a chair normally, and get up from a chair normally',
    '14': 'Sit down to a chair quickly, and get up from a chair quickly',
    '15': 'Sit a moment, trying to get up, and collapse into a chair',
    '16': 'Stand, sit on the sofa (back is inclined to the support), and get up normally',
    '17': 'Lie on the bed for 30 seconds',
    '18': 'Sit a moment, lie down to the bed normally, and get up normally',
    '19': 'Sit a moment, lie down to the bed quickly, and get up quickly',
    '20': 'Walk upstairs and downstairs normally (5 steps)',
    '21': 'Walk upstairs and downstairs quickly (5 steps)',
    '22': 'Forward fall when trying to sit down',
    '23': 'Backward fall when trying to sit down',
    '24': 'Lateral fall when trying to sit down',
    '25': 'Forward fall when trying to get up',
    '26': 'Lateral fall when trying to get up',
    '27': 'Forward fall while sitting, caused by fainting',
    '28': 'Lateral fall while sitting, caused by fainting',
    '29': 'Backward fall while sitting, caused by fainting',
    '30': 'Vertical(forward) fall while walking caused by fainting',
    '31': 'Fall while walking, use of hands to dampen fall, caused by fainting',
    '32': 'Forward fall while walking caused by a trip',
    '33': 'Forward fall while jogging caused by a trip',
    '34': 'Forward fall while walking caused by a slip',
    '35': 'Lateral fall while walking caused by a slip',
    '36': 'Backward fall while walking caused by a slip'
}


# 센서 데이터에 라벨 열 추가
sensor_data['Label'] = sensor_data['Task ID'].map(label_dict)

print(sensor_data)

         TimeStamp(s)  FrameCounter   AccX   AccY   AccZ      GyrX      GyrY  \
0                0.01             1 -0.053 -1.004  0.000 -0.057296  0.401071   
1                0.02             2 -0.054 -1.007  0.000 -0.057296  0.229183   
2                0.03             3 -0.056 -1.009 -0.004 -0.458366  0.286479   
3                0.04             4 -0.053 -1.011 -0.009 -0.916733  0.458366   
4                0.05             5 -0.052 -1.012 -0.011 -1.088620  0.343775   
...               ...           ...    ...    ...    ...       ...       ...   
3995095          8.16           816 -0.035 -1.004  0.138 -0.229183 -1.661578   
3995096          8.17           817 -0.030 -1.000  0.135 -0.630254 -1.375099   
3995097          8.18           818 -0.024 -0.997  0.131 -1.203212 -1.546987   
3995098          8.19           819 -0.019 -0.997  0.126 -1.776170 -1.776170   
3995099          8.20           820 -0.014 -0.999  0.120 -2.234536 -2.234536   

             GyrZ     EulerX    EulerY 

In [16]:
import pandas as pd

filename = '../data/sensor_data_label.csv'

# 데이터프레임을 CSV 파일로 저장
sensor_data.to_csv(filename, index=False)

print(f"데이터가 {filename} 파일로 저장되었습니다.")

데이터가 ../data/sensor_data_label.csv 파일로 저장되었습니다.


In [17]:
print(sensor_data.columns)

Index(['TimeStamp(s)', 'FrameCounter', 'AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY',
       'GyrZ', 'EulerX', 'EulerY', 'EulerZ', 'Subject ID', 'Task ID',
       'Trial ID', 'Label'],
      dtype='object')


#### 필드 정보
- 'TimeStamp(s)': 시간 스탬프 (측정 시간)
- 'FrameCounter': 프레임 카운터 (데이터 포인트의 순서 또는 인덱스)
- 가속도 (Acceleration)는 물체의 움직임의 변화율을 나타내며, 센서가 감지하는 선형 가속도의 세 가지 축(X, Y, Z)에 대한 값. 이 값은 일반적으로 "g"라는 단위로 표현되며, 중력 가속도의 배수로 표현될 수 있다.
'AccX': X축 가속도
'AccY': Y축 가속도
'AccZ': Z축 가속도


- 자이로스코프 (Gyroscope)는 회전 속도를 측정하는 센서로, 물체의 회전에 대한 정보를 제공. 센서가 감지하는 회전 속도는 X, Y, Z축 각각에 대한 값으로 표현. 일반적으로 "°/s"라는 단위로 표시.
'GyrX': X축 자이로스코프 (회전 속도)
'GyrY': Y축 자이로스코프 (회전 속도)
'GyrZ': Z축 자이로스코프 (회전 속도)

- 오일러 각도 (Euler Angle)는 물체의 방향을 나타내는 값으로, 세 개의 축 (롤, 피치, 요우)을 사용하여 표현. 'EulerX', 'EulerY', 'EulerZ'는 각각 X, Y, Z축에 대한 오일러 각도를 나타낸다. 이 값은 일반적으로 "°"로 표시된다.
'EulerX': X축 오일러 각도
'EulerY': Y축 오일러 각도
'EulerZ': Z축 오일러 각도
- 'Subject ID': 주체 ID (데이터 수집 대상의 고유 식별자)
- 'Task ID': 작업 ID (수행한 작업의 유형 또는 번호)
- 'Trial ID': 시도 ID (해당 작업의 시도 번호)
- 'Label': 레이블 (데이터 포인트의 분류 또는 카테고리)